In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import seaborn as sns
from glob import glob
from tqdm import tqdm
from google.colab import drive
from scipy.ndimage import center_of_mass
from PIL import Image
from matplotlib import pyplot as plt
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Change these two lines each time testing new model
# use_all = False means we don't test on augmented images
# (only on rotated images)
model_name = "model"
use_all = False

# Init paths
dataset_path = '/content/drive/MyDrive/Images/test'
save_path = f'/content/drive/MyDrive/Predictions/{model_name}'
model_path = f'/content/drive/MyDrive/Files/models/{model_name}.h5'
os.makedirs(save_path, exist_ok=True)

# Random seed
os.environ['PYTHONHASHSEED'] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

# Load model & test images
model = tf.keras.models.load_model(model_path, compile=False)
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=["acc", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
test_x = sorted(glob(os.path.join(dataset_path, 'images', '*')))
pred = sorted(glob(os.path.join(save_path, '*')))

In [ ]:
#---------------------------SAVING MODEL PREDICTIONS----------------------------
expected = len(test_x)
if len(pred) < expected:
  for x in tqdm(test_x):
    name = x.split('/')[-1]

    x = cv2.imread(x, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (224,224))
    x = x / 255.0
    x = np.expand_dims(x, axis=0)

    with tf.device(tf.test.gpu_device_name()):
      p = model.predict(x, verbose=0)[0]

    p = p > 0.5 #thresholding
    p = p * 255

    cv2.imwrite(os.path.join(save_path, name), p)

In [ ]:
#---------------------------SAVING CONFUSION MATRIX-----------------------------
FP, FN, TP, TN = 0, 0, 0, 0

y_actual = sorted(glob(os.path.join(dataset_path, 'masks', '*')))
target, append = 256, 'all_images'
if not use_all:
  target, append = 55, 'only_rotated'
  y_actual = [f for f in y_actual if 'rotated' in f]
print(f'{len(y_actual)} test images loaded (should be {target})')

for y in tqdm(y_actual):
  name = y.split('/')[-1]
  name = name[:-3] + 'jpg'

  def im_read(im):
    im = cv2.imread(im, cv2.IMREAD_GRAYSCALE)
    im = cv2.resize(im, (224,224))
    im = im / 255.0
    im = im > 0.5
    return im

  y = im_read(y)
  y_pred = im_read(os.path.join(save_path, name))

  FP = FP + len(np.where(y_pred - y  == 1)[0])
  FN = FN + len(np.where(y_pred - y  == -1)[0])
  TP = TP + len(np.where(y_pred + y == 2)[0])
  TN = TN + len(np.where(y_pred + y == 0)[0])

os.makedirs(f"/content/drive/MyDrive/Comparing Models/{model_name}/", exist_ok=True)
with open(f"/content/drive/MyDrive/Comparing Models/{model_name}/metrics_{append}.txt", "w") as f:
  f.write(f"false pos: {FP}")
  f.write(f"\nfalse neg: {FN}")
  f.write(f"\ntrue pos: {TP}")
  f.write(f"\ntrue neg: {TN}")
  f.write(f"\nsum (total pixels): {(TP + TN + FP + FN)}\n")
  f.write(f"\ncorrect pos out of all positives (precision): {TP / (TP + FP)}")
  f.write(f"\ncorrect neg out of all negatives (recall): {TP / (TP + FN)}")
  f.write(f"\ncorrect predictions out of all (accuracy): {(TP + TN) / (TP + TN + FP + FN)}")

cmat = [[TN, FP], [FN, TP]]
plt.figure(figsize = (6,6))
sns.heatmap(cmat/np.sum(cmat), cmap="Reds", annot=True, fmt = '.2%', square=1,   linewidth=2.)
plt.xlabel("predictions")
plt.ylabel("actual mask")
plt.savefig(f'/content/drive/MyDrive/Comparing Models/{model_name}/matrix_{append}.jpg')
plt.show()